# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 65 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.07324


extracting tarball to tmp_2304.07324...

 done.
Retrieving document from  https://arxiv.org/e-print/2304.07720


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.07324/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.


Found 62 bibliographic references in tmp_2304.07720/wg15_2022.bbl.
syntax error in line 151: '=' expected
Retrieving document from  https://arxiv.org/e-print/2304.07793


extracting tarball to tmp_2304.07793...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.07793/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'conclusion.tex' from 'tmp_2304.07793/conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'results.tex' from 'tmp_2304.07793/results.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.07720-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.07720) | **The Gaia-ESO Survey: homogenisation of stellar parameters and elemental  abundances**  |
|| A. Hourihane, et al. -- incl., <mark>M. Bergemann</mark>, <mark>G. Guiglion</mark> |
|*Appeared on*| *2023-04-18*|
|*Comments*| *A&A accepted, minor revision, 36 pages, 38 figures*|
|**Abstract**| The Gaia-ESO Survey is a public spectroscopic survey that has targeted $\gtrsim10^5$ stars covering all major components of the Milky Way from the end of 2011 to 2018, delivering its public final release in May 2022. Unlike other spectroscopic surveys, Gaia-ESO is the only survey that observed stars across all spectral types with dedicated, specialised analyses: from O ($T_\mathrm{eff} \sim 30,000-52,000$~K) all the way to K-M ($\gtrsim$3,500~K). The physics throughout these stellar regimes varies significantly, which has previously prohibited any detailed comparisons between stars of significantly different type. In the final data release (internal data release 6) of the Gaia-ESO Survey, we provide the final database containing a large number of products such as radial velocities, stellar parameters and elemental abundances, rotational velocity, and also, e.g., activity and accretion indicators in young stars and membership probability in star clusters for more than 114,000 stars. The spectral analysis is coordinated by a number of Working Groups (WGs) within the Survey, which specialise in the various stellar samples. Common targets are analysed across WGs to allow for comparisons (and calibrations) amongst instrumental setups and spectral types. Here we describe the procedures employed to ensure all Survey results are placed on a common scale to arrive at a single set of recommended results for all Survey collaborators to use. We also present some general quality and consistency checks performed over all Survey results. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.07324-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.07324) | **Spectroscopic follow-up of black hole and neutron star candidates in  ellipsoidal variables from Gaia DR3**  |
|| P. Nagarajan, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2023-04-18*|
|*Comments*| *18 pages, 12 figures*|
|**Abstract**| We present multi-epoch spectroscopic follow-up of a sample of ellipsoidal variables selected from Gaia DR3 as candidates for hosting quiescent black holes (BHs) and neutron stars (NSs). Our targets were identified as BH/NS candidates because their optical light curves -- when interpreted with models that attribute variability to tidal distortion of a star by a companion that contributes negligible light -- suggest that the companions are compact objects. From the likely BH/NS candidates identified in recent work accompanying Gaia DR3, we select 14 of the most promising targets for follow-up. We obtained spectra for each object at 2-10 epochs, strategically observing near conjunction to best-constrain the radial velocity semi-amplitude. From the measured semi-amplitudes of the radial velocity curves, we derive minimum companion masses of $M_{2,\min} \leq 0.5 ~ M_{\odot}$ in all cases. Assuming random inclinations, the typical inferred companion mass is $M_2 \sim 0.15 ~ M_{\odot}$. This makes it unlikely that any of these systems contain a BH or NS, and we consider alternative explanations for the observed variability. We can best reproduce the observed light curves and radial velocities with models for unequal-mass contact binaries with starspots. Some of the objects in our sample may also be detached main-sequence binaries, or even single stars with pulsations or starspot variability masquerading as ellipsoidal variation. We provide recommendations for future spectroscopic efforts to further characterize this sample and more generally to search for compact object companions in close binaries. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.07793-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.07793) | **Dark matter search with CMB: a study of foregrounds**  |
|| <mark>Z.-X. Zhang</mark>, et al. |
|*Appeared on*| *2023-04-18*|
|*Comments*| *6 figures, 2 tables. The foreground, mask maps and simulated datasets used in this work are available at this https URL*|
|**Abstract**| The energy injected from dark matter annihilation and decay processes potentially raises the ionisation of the intergalactic medium and leaves visible footprints on the anisotropy maps of the cosmic microwave background (CMB). Galactic foregrounds emission in the microwave bands contaminate the CMB measurement and may affect the search for dark matter's signature. In this paper, we construct a full CMB data and foreground simulation based on the design of the next-generation ground-based CMB experiments. The foreground residual after the components separation on maps is fully considered in our data analysis, accounting for various contamination from the emission of synchrotron, thermal dust, free-free and spinning dust. We analyse the corresponding sensitivity on dark matter parameters from the temperature and polarization maps, and we find that the CMB foregrounds leave a non-zero yet controllable impact on the sensitivity. Comparing with statistics-only analysis, the CMB foreground residual leads to a factor of 7%-23% weakening on energy-injection constraints, depending on the specific dark matter process and experimental configuration. Strong limits on dark matter annihilation rate and decay lifetime can be expected after foreground subtraction. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.07720.md
    + _build/html/tmp_2304.07720/./worley_gesdataflow_feb2023.png
    + _build/html/tmp_2304.07720/./wg15_rv_workflow.png
    + _build/html/tmp_2304.07720/./WG15_dataflow_aph.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\feh}{\mathrm{[Fe/H]}}$
$\newcommand{\vsini}{\ensuremath{v\sin i}}$
$\newcommand{\vrad}{V_\mathrm{rad}}$</div>



<div id="title">

# The Gaia-ESO Survey: homogenisation of stellar parameters and elemental abundances 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.07720-b31b1b.svg)](https://arxiv.org/abs/2304.07720)<mark>Appeared on: 2023-04-18</mark> -  _A&A accepted, minor revision, 36 pages, 38 figures_

</div>
<div id="authors">

A. Hourihane, et al. -- incl., <mark>M. Bergemann</mark>, <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** The Gaia-ESO Survey is a public spectroscopic survey that has targeted $\gtrsim10^5$ stars covering all major components of the Milky Way from the end of 2011 to 2018, delivering its public final release in May 2022. Unlike other spectroscopic surveys, Gaia-ESO is the only survey that observed stars across all spectral types with dedicated, specialised analyses: from O ( $T_\mathrm{eff} \sim 30,000-52,000$ K) all the way to K-M ( $\gtrsim$ 3,500 K). The physics throughout these stellar regimes varies significantly, which has previously prohibited any detailed comparisons between stars of significantly different type. In the final data release (internal data release 6) of the Gaia-ESO Survey, we provide the final database containing a large number of products such as radial velocities, stellar parameters and elemental abundances, rotational velocity, and also, e.g.,  activity and accretion indicators in young stars and membership probability in star clusters  for more than 114,000 stars. The spectral analysis is coordinated by a number of Working Groups (WGs) within the Survey, which specialise in the various stellar samples. Common targets are analysed across WGs to allow for comparisons (and calibrations) amongst instrumental setups and spectral types. Here we describe the procedures employed to ensure all Survey results are placed on a common scale to arrive at a single set of recommended results for all Survey collaborators to use. We also present some general quality and consistency checks performed over all Survey results.

</div>

<div id="div_fig1">

<img src="tmp_2304.07720/./worley_gesdataflow_feb2023.png" alt="Fig1" width="100%"/>

**Figure 1. -** Gaia-ESO data flow diagram for iDR6 showing the key stages from target selection to data release via the archives. (*fig:idr6_dataflow_ppt*)

</div>
<div id="div_fig2">

<img src="tmp_2304.07720/./wg15_rv_workflow.png" alt="Fig7" width="100%"/>

**Figure 7. -** Schematic view of the GES WG15 Radial Velocity Homogenisation Work-Flow. (*fig:rvhomg_wrkflw*)

</div>
<div id="div_fig3">

<img src="tmp_2304.07720/./WG15_dataflow_aph.png" alt="Fig11" width="100%"/>

**Figure 11. -** Gaia-ESO data processing diagram for iDR6 showing the complexity of the interfaces between the reduction, parameter, abundance and homogenisation processes. The GRR is the GES Results Repository at CASU. (*fig:idr6_dataflow_yed*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.07720"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

190  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6

  6 publications selected.
